In [1]:
import numpy as np
import json
import os
import copy
import pickle
import trimesh

from torch.utils.data import DataLoader, Dataset

import torch
from tensorboardX import SummaryWriter

from sklearn.metrics.pairwise import euclidean_distances

root_dir = '/home/jingwang/Data/data/'

dataset = 'FaceWarehouse'
name = 'neutral_140_10_id'

torch.backends.cudnn.benchmark = True


GPU = True
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
device_idx = 0
print(torch.cuda.get_device_name(device_idx))

GeForce GTX TITAN X


In [2]:
generative_model = 'autoencoder_2FC'
meshpackage = 'trimesh'

reference_mesh_file = os.path.join(root_dir, dataset, 'template', 'template.obj')

args = {'generative_model': generative_model,
        'name': name, 'data': os.path.join(root_dir, dataset, 'preprocessed',name),
        'results_folder':  os.path.join(root_dir, dataset,'results/fully_connected_'+ generative_model),
        'checkpoint_file': 'checkpoint',
        'seed':2, 'loss':'l1',
        'batch_size': 1, 'num_epochs':1000, 'eval_frequency':200, 'num_workers': 40,
        'nz': 128,
        
        'lr':1e-4, 
        'regularization': 5e-4,         
        'scheduler': True, 'decay_rate': 0.99, 'decay_steps': 1000,  
        'resume': False,
        'reference_mesh_file': reference_mesh_file,
        'mode':'train', 'shuffle': True, 'nVal': 1, 'normalization': True,
        'bs_write_mesh': True,
        'worst_face_num': 20}

args['results_folder'] = os.path.join(args['results_folder'],'latent_'+str(args['nz']))
    
if not os.path.exists(os.path.join(args['results_folder'])):
    os.makedirs(os.path.join(args['results_folder']))

summary_path = os.path.join(args['results_folder'],'summaries',args['name'])
if not os.path.exists(summary_path):
    os.makedirs(summary_path)  
    
checkpoint_path = os.path.join(args['results_folder'],'checkpoints', args['name'])
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
    
samples_path = os.path.join(args['results_folder'],'samples', args['name'])
if not os.path.exists(samples_path):
    os.makedirs(samples_path)
    
prediction_path = os.path.join(args['results_folder'],'predictions', args['name'])
if not os.path.exists(prediction_path):
    os.makedirs(prediction_path)

downsample_mesh_path = os.path.join(args['results_folder'],'downsample_mesh', args['name'])
if not os.path.exists(downsample_mesh_path):
    os.makedirs(downsample_mesh_path)


worst_mesh_path = os.path.join(args['results_folder'],'worst_test', args['name'])
if not os.path.exists(downsample_mesh_path):
    os.makedirs(downsample_mesh_path)

In [3]:
torch.manual_seed(args['seed'])

if GPU:
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    device = torch.device("cuda:"+str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

cuda:0


In [4]:
class OnDeviceDataset(Dataset):
    def __init__(self, np_data, shapedata, device):
        self.shapedata = shapedata
        self.data = torch.as_tensor(np_data,device=device)
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        points = self.data[idx]
        return {
            'points': points
        }

In [5]:

### Code obtained and modified from https://github.com/anuragranj/coma, Copyright (c) 2018 Anurag Ranjan, Timo Bolkart, Soubhik Sanyal, Michael J. Black and the Max Planck Gesellschaft

import os
import numpy as np

from trimesh.exchange.export import export_mesh
import trimesh

import time
from tqdm import tqdm

class ShapeData(object):
    def __init__(self, nVal, train_file, test_file, reference_mesh_file, normalization = True, meshpackage = 'mpi-mesh', load_flag = True, mean_subtraction_only = False):
        self.nVal = nVal
        self.train_file = train_file
        self.test_file = test_file
        self.vertices_train = None
        self.vertices_val = None
        self.vertices_test = None
        self.n_vertex = None
        self.n_features = None
        self.normalization = normalization
        self.meshpackage = meshpackage
        self.load_flag = load_flag
        self.mean_subtraction_only = mean_subtraction_only
        
        if self.load_flag:
            self.load()
        if self.meshpackage == 'trimesh':
            self.reference_mesh = trimesh.load(reference_mesh_file, process = False)
        elif self.meshpackage =='mpi-mesh':
            self.reference_mesh = Mesh(filename=reference_mesh_file)
        
        if self.load_flag:
            self.mean = np.mean(self.vertices_train, axis=0)
            self.std = np.std(self.vertices_train, axis=0)
        else:
            self.mean = None
            self.std = None
        self.normalize()
        
    def load(self):
        vertices_train = np.load(self.train_file)
        self.vertices_train = vertices_train[:-self.nVal]
        self.vertices_val = vertices_train[-self.nVal:]

        self.n_vertex = self.vertices_train.shape[1]
        self.n_features = self.vertices_train.shape[2]

        if os.path.exists(self.test_file):
            self.vertices_test = np.load(self.test_file)
            self.vertices_test = self.vertices_test

    def normalize(self):
        if self.load_flag:
            if self.normalization:
                if self.mean_subtraction_only:
                    self.std = np.ones_like((self.std))
                
                if self.vertices_train.shape[0] > 0:
                    self.vertices_train = self.vertices_train - self.mean
                    self.vertices_train = self.vertices_train/self.std
                    self.vertices_train[np.where(np.isnan(self.vertices_train))]=0.0

                if self.vertices_val.shape[0] > 0:
                    self.vertices_val = self.vertices_val - self.mean
                    self.vertices_val = self.vertices_val/self.std
                    self.vertices_val[np.where(np.isnan(self.vertices_val))]=0.0

                if self.vertices_test.shape[0] > 0: # not empty
                    self.vertices_test = self.vertices_test - self.mean
                    self.vertices_test = self.vertices_test/self.std
                    self.vertices_test[np.where(np.isnan(self.vertices_test))]=0.0
                
                self.N = self.vertices_train.shape[0]

                print('Vertices normalized')
            else:
                print('Vertices not normalized')


    def save_meshes(self, filename, meshes, mesh_indices):
        for i in range(meshes.shape[0]):
            if self.normalization:
                vertices = meshes[i].reshape((self.n_vertex, self.n_features))*self.std + self.mean
            else:
                vertices = meshes[i].reshape((self.n_vertex, self.n_features))
            if self.meshpackage == 'trimesh':
                new_mesh = self.reference_mesh
                if self.n_features == 3:
                    new_mesh.vertices = vertices
                elif self.n_features == 6:
                    new_mesh.vertices = vertices[:,0:3]
                    colors = vertices[:,3:]
                    colors[np.where(colors<0)]=0
                    colors[np.where(colors>1)]=1
                    vertices[:,3:] = colors
                    new_mesh.visual = trimesh.visual.create_visual(vertex_colors = vertices[:,3:])
                else:
                    raise NotImplementedError
                new_mesh.export(filename+'.'+str(mesh_indices[i]).zfill(6)+'.ply','ply')   
            elif self.meshpackage =='mpi-mesh':
                if self.n_features == 3:
                    mesh = Mesh(v=vertices, f=self.reference_mesh.f)
                    mesh.write_ply(filename+'.'+str(mesh_indices[i]).zfill(6)+'.ply')
                else:
                    raise NotImplementedError
        return 0


In [6]:
np.random.seed(args['seed'])
print("Loading data .. ")
if True: # not os.path.exists(args['data']+'/mean.npy') or not os.path.exists(args['data']+'/std.npy'):
    shapedata =  ShapeData(nVal=args['nVal'], 
                          train_file=args['data']+'/train.npy', 
                          test_file=args['data']+'/test.npy', 
                          reference_mesh_file=args['reference_mesh_file'],
                          normalization = args['normalization'],
                          meshpackage = meshpackage, load_flag = True)
    np.save(args['data']+'/mean.npy', shapedata.mean)
    np.save(args['data']+'/std.npy', shapedata.std)
else:
    shapedata = ShapeData(nVal=args['nVal'], 
                         train_file=args['data']+'/train.npy',
                         test_file=args['data']+'/test.npy', 
                         reference_mesh_file=args['reference_mesh_file'],
                         normalization = args['normalization'],
                         meshpackage = meshpackage, load_flag = True)
    shapedata.mean = np.load(args['data']+'/mean.npy')
    shapedata.std = np.load(args['data']+'/std.npy')
    shapedata.n_vertex = shapedata.mean.shape[0]
    shapedata.n_features = shapedata.mean.shape[1]

specified material ()  not loaded!


Loading data .. 
Vertices normalized


In [7]:
# Building model, optimizer, and loss function

dataset_train = OnDeviceDataset(shapedata.vertices_train, shapedata, device)

dataloader_train = DataLoader(dataset_train, batch_size=args['batch_size'],\
                                     shuffle = args['shuffle'])

dataset_val = OnDeviceDataset(shapedata.vertices_val, shapedata, device)

dataloader_val = DataLoader(dataset_val, batch_size=args['batch_size'],\
                                     shuffle = False)

dataset_test = OnDeviceDataset(shapedata.vertices_test, shapedata, device)

dataloader_test = DataLoader(dataset_test, batch_size=args['batch_size'],\
                                     shuffle = False)

if 'autoencoder_2FC' == args['generative_model']:
    import torch.nn as nn
    class MLPAutoEncoder(nn.Module):
        def __init__(self, nz):
            super().__init__()
            self.nz = nz
            self.en_fc1 = nn.Linear(11510*3, 1024)
            self.en_fc2 = nn.Linear(1024, nz)
            self.de_fc1 = nn.Linear(nz, 1024)
            self.de_fc2 = nn.Linear(1024, 11510*3)
            self._only_encode = False
            self._only_decode = False
            self.activation = nn.ELU()
        
        def only_encode(self, status):
            self._only_encode = status
        
        def only_decode(self, status):
            self._only_decode = status
        
        def encode(self, x):
            x = x.reshape(x.shape[0],-1)
            x = self.en_fc1(x)
            x = self.activation(x)
            x = self.en_fc2(x)
            return x
        
        def decode(self, x):
            x = self.de_fc1(x)
            x = self.activation(x)
            x = self.de_fc2(x)
            x = x.reshape(x.shape[0],-1,3)
            return x
        
        def forward(self, x):
            if self._only_decode:
                x = self.decode(x)
                return x
            z = self.encode(x)
            if self._only_encode:
                return z
            x = self.decode(z)
            return x
            
  
    model = MLPAutoEncoder(nz=args['nz']).to(device)
    
optim = torch.optim.Adam(model.parameters(),lr=args['lr'],weight_decay=args['regularization'])
if args['scheduler']:
    scheduler=torch.optim.lr_scheduler.StepLR(optim, args['decay_steps'],gamma=args['decay_rate'])
else:
    scheduler = None
    
if args['loss']=='l1':
    def loss_l1(outputs, targets):
        L = torch.abs(outputs - targets).mean()
        return L 
    loss_fn = loss_l1

In [8]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of parameters is: {}".format(params)) 
print(model)
# print(M[4].v.shape)

Total number of parameters is: 71016290
MLPAutoEncoder(
  (en_fc1): Linear(in_features=34530, out_features=1024, bias=True)
  (en_fc2): Linear(in_features=1024, out_features=128, bias=True)
  (de_fc1): Linear(in_features=128, out_features=1024, bias=True)
  (de_fc2): Linear(in_features=1024, out_features=34530, bias=True)
  (activation): ELU(alpha=1.0)
)


In [9]:
def train_autoencoder_dataloader(dataloader_train, dataloader_val,
                                 device, model, optim, loss_fn, 
                                 bsize, start_epoch, n_epochs, eval_freq, scheduler = None,
                                 writer=None, save_recons=True, shapedata = None,
                                 metadata_dir=None, samples_dir = None, checkpoint_path = None):
    if not shapedata.normalization:
        shapedata_mean = torch.Tensor(shapedata.mean).to(device)
        shapedata_std = torch.Tensor(shapedata.std).to(device)
    
    total_steps = start_epoch*len(dataloader_train)

    for epoch in range(start_epoch, n_epochs):
        model.train()

        tloss = []
        for b, sample_dict in enumerate(tqdm(dataloader_train)):
            optim.zero_grad()
                
            tx = sample_dict['points'].to(device)
            cur_bsize = tx.shape[0]
            
            tx_hat = model(tx)
            loss = loss_fn(tx, tx_hat)

            loss.backward()
            optim.step()
            
            if shapedata.normalization:
                tloss.append(cur_bsize * loss.item())
            else:
                with torch.no_grad():
                    tx_norm = tx
                    tx_hat_norm = tx_hat
                    tx_norm = (tx_norm - shapedata_mean)/shapedata_std
                    tx_norm = torch.cat((tx_norm,torch.zeros(tx.shape[0],1,tx.shape[2]).to(device)),1)
                    
                    tx_hat_norm = (tx_hat_norm -shapedata_mean)/shapedata_std
                    tx_hat_norm = torch.cat((tx_hat_norm,torch.zeros(tx.shape[0],1,tx.shape[2]).to(device)),1)
                    
                    loss_norm = loss_fn(tx_norm, tx_hat_norm)
                    tloss.append(cur_bsize * loss_norm.item())
            if writer and total_steps % eval_freq == 0:
                writer.add_scalar('loss/loss/data_loss',loss.item(),total_steps)
                writer.add_scalar('training/learning_rate', optim.param_groups[0]['lr'],total_steps)
            total_steps += 1

        # validate
        model.eval()
        vloss = []
        with torch.no_grad():
            for b, sample_dict in enumerate(tqdm(dataloader_val)):

                tx = sample_dict['points'].to(device)
                cur_bsize = tx.shape[0]

                tx_hat = model(tx)
                loss = loss_fn(tx, tx_hat)
                
                if shapedata.normalization:
                    vloss.append(cur_bsize * loss.item())
                else:
                    with torch.no_grad():
                        if shapedata.mean.shape[0]!=tx.shape[1]:
                            tx_norm = tx[:,:-1,:]
                            tx_hat_norm = tx_hat[:,:-1,:]
                        else:
                            tx_norm = tx
                            tx_hat_norm = tx_hat
                        tx_norm = (tx_norm - shapedata_mean)/shapedata_std
                        tx_norm = torch.cat((tx_norm,torch.zeros(tx.shape[0],1,tx.shape[2]).to(device)),1)
                    
                        tx_hat_norm = (tx_hat_norm - shapedata_mean)/shapedata_std
                        tx_hat_norm = torch.cat((tx_hat_norm,torch.zeros(tx.shape[0],1,tx.shape[2]).to(device)),1)
                    
                        loss_norm = loss_fn(tx_norm, tx_hat_norm)
                        vloss.append(cur_bsize * loss_norm.item())   

        if scheduler:
            scheduler.step()
            
        epoch_tloss = sum(tloss) / float(len(dataloader_train.dataset))
        writer.add_scalar('avg_epoch_train_loss',epoch_tloss,epoch)
        if len(dataloader_val.dataset) > 0:
            epoch_vloss = sum(vloss) / float(len(dataloader_val.dataset))
            writer.add_scalar('avg_epoch_valid_loss', epoch_vloss,epoch)
            print('epoch {0} | tr {1} | val {2}'.format(epoch,epoch_tloss,epoch_vloss))
        else:
            print('epoch {0} | tr {1} '.format(epoch,epoch_tloss))
        
        model.cpu()
  
        torch.save({'epoch': epoch,
            'autoencoder_state_dict': model.state_dict(),
            'optimizer_state_dict' : optim.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
        },os.path.join(metadata_dir, checkpoint_path+'.pth.tar'))
        
        if epoch % 10 == 0:
            torch.save({'epoch': epoch,
            'autoencoder_state_dict': model.state_dict(),
            'optimizer_state_dict' : optim.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            },os.path.join(metadata_dir, checkpoint_path+'%s.pth.tar'%(epoch)))
        
        model.to(device)

        if save_recons:
            with torch.no_grad():
                if epoch == 0:
                    mesh_ind = [0]
                    msh = tx[mesh_ind].detach().cpu().numpy()
                    shapedata.save_meshes(os.path.join(samples_dir,'input_epoch_{0}'.format(epoch)),
                                                     msh, mesh_ind)
                mesh_ind = [0]
                msh = tx_hat[mesh_ind].detach().cpu().numpy()
                shapedata.save_meshes(os.path.join(samples_dir,'epoch_{0}'.format(epoch)),
                                                 msh, mesh_ind)

    print('~FIN~')

In [10]:
if args['mode'] == 'train':
    writer = SummaryWriter(summary_path)
    with open(os.path.join(args['results_folder'],'checkpoints', args['name'] +'_params.json'),'w') as fp:
        saveparams = copy.deepcopy(args)
        json.dump(saveparams, fp)
        
    if args['resume']:
            print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file'])))
            checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
            start_epoch = checkpoint_dict['epoch'] + 1
            model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
            optim.load_state_dict(checkpoint_dict['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint_dict['scheduler_state_dict'])
            print('Resuming from epoch %s'%(str(start_epoch)))     
    else:
        start_epoch = 0
        
    train_autoencoder_dataloader(dataloader_train, dataloader_test, # fixme dataloader_val
                      device, model, optim, loss_fn,
                      bsize = args['batch_size'],
                      start_epoch = start_epoch,
                      n_epochs = args['num_epochs'],
                      eval_freq = args['eval_frequency'],
                      scheduler = scheduler,
                      writer = writer,
                      save_recons=True,
                      shapedata=shapedata,
                      metadata_dir=checkpoint_path, samples_dir=samples_path,
                      checkpoint_path = args['checkpoint_file'])

100%|██████████| 10/10 [00:00<00:00, 764.12it/s]


epoch 0 | tr 0.7470248920454395 | val 0.7579585492610932


100%|██████████| 10/10 [00:00<00:00, 743.33it/s]


epoch 1 | tr 0.5682690621708795 | val 0.5860979616641998


100%|██████████| 10/10 [00:00<00:00, 759.81it/s]


...

epoch 997 | tr 0.10624212149879057 | val 0.32590864449739454


100%|██████████| 10/10 [00:00<00:00, 758.56it/s]


epoch 998 | tr 0.1050861382977568 | val 0.33294921517372134


100%|██████████| 10/10 [00:00<00:00, 756.03it/s]


epoch 999 | tr 0.10773994387696972 | val 0.3258523404598236
~FIN~


In [35]:
import torch
import copy
from tqdm import tqdm
import numpy as np
import os
import heapq

def test_autoencoder_dataloader(device, model, dataloader_test, shapedata, worst_face_num, worst_path, mm_constant = 1000, save_recons=True, samples_dir=None):
    model.eval()
    l1_loss = 0
    l2_loss = 0
    shapedata_mean = torch.Tensor(shapedata.mean).to(device)
    shapedata_std = torch.Tensor(shapedata.std).to(device)

    min_heap = []

    with torch.no_grad():
        for i, sample_dict in enumerate(tqdm(dataloader_test)):
            tx = sample_dict['points'].to(device)
            prediction = model(tx)  
            if i==0:
                predictions = copy.deepcopy(prediction)
            else:
                predictions = torch.cat([predictions,prediction],0) 
                
            if dataloader_test.dataset.dummy_node:
                x_recon = prediction[:,:-1]
                x = tx[:,:-1]
            else:
                x_recon = prediction
                x = tx
            l1_loss+= torch.mean(torch.abs(x_recon-x))*x.shape[0]/float(len(dataloader_test.dataset))
            
            old_x_recon = x_recon
            old_x = x

            x_recon = (x_recon * shapedata_std + shapedata_mean) * mm_constant
            x = (x * shapedata_std + shapedata_mean) * mm_constant
            per_l2_loss = torch.sqrt(torch.sum((x_recon - x)**2,dim=2))

            for j in range(per_l2_loss.shape[0]):
                payload = (torch.mean(per_l2_loss[j]), i *
                           per_l2_loss.shape[0]+j, old_x[j], old_x_recon[j])
                # print(payload[2].shape, payload[3].shape)
                # print('pushing',payload)
                if len(min_heap) < worst_face_num:
                    heapq.heappush(min_heap, payload)
                else:
                    heapq.heappushpop(min_heap, payload)
            l2_loss+= torch.mean(per_l2_loss)*x.shape[0]/float(len(dataloader_test.dataset))

        msh = []
        msh_recon = []
        msh_ind = []
        for loss, idx, x, x_recon in min_heap:
            print('loss for ', idx, loss)
            msh_ind.append(idx)
            msh.append(x.detach().cpu().numpy())
            msh_recon.append(x_recon.detach().cpu().numpy())
        msh = np.array(msh)
        msh_recon = np.array(msh_recon) # prevbug msh_ind msh_recon
        shapedata.save_meshes(os.path.join(worst_path, 'input'), msh, msh_ind)
        shapedata.save_meshes(os.path.join(
            worst_path, 'worst'), msh_recon, msh_ind)

            
        predictions = predictions.cpu().numpy()
        l1_loss = l1_loss.item()
        l2_loss = l2_loss.item()

    return predictions, l1_loss, l2_loss


In [17]:
if args['mode'] == 'test':
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    predictions, norm_l1_loss, l2_loss = test_autoencoder_dataloader(device, model, dataloader_test, 
                                                                     shapedata, worst_path=worst_mesh_path, 
                                                                     worst_face_num=args['worst_face_num'],
                                                                     mm_constant = 100)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)   
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

loading checkpoint from file /home/jingwang/Data/data/FaceWarehouse/results/spirals_simple_autoencoder/latent_128/checkpoints/align_pose/checkpoint.pth.tar


100%|██████████| 21/21 [00:02<00:00,  8.53it/s]


('loss for ', 612, tensor(4.3394, device='cuda:0'))
('loss for ', 602, tensor(4.3528, device='cuda:0'))
('loss for ', 194, tensor(4.3574, device='cuda:0'))
('loss for ', 560, tensor(4.3619, device='cuda:0'))
('loss for ', 390, tensor(4.4021, device='cuda:0'))
('loss for ', 402, tensor(4.3611, device='cuda:0'))
('loss for ', 537, tensor(4.4436, device='cuda:0'))
('loss for ', 625, tensor(4.3799, device='cuda:0'))
('loss for ', 8, tensor(4.4154, device='cuda:0'))
('loss for ', 628, tensor(4.5263, device='cuda:0'))
('loss for ', 559, tensor(4.6070, device='cuda:0'))
('loss for ', 626, tensor(4.6145, device='cuda:0'))
('loss for ', 370, tensor(4.4514, device='cuda:0'))
('loss for ', 125, tensor(4.5456, device='cuda:0'))
('loss for ', 389, tensor(4.4977, device='cuda:0'))
('loss for ', 364, tensor(4.5176, device='cuda:0'))
('loss for ', 169, tensor(4.5499, device='cuda:0'))
('loss for ', 499, tensor(4.4825, device='cuda:0'))
('loss for ', 341, tensor(4.4610, device='cuda:0'))
('loss for ', 

In [7]:
if args['mode'] == 'test': # test with train set
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    predictions, norm_l1_loss, l2_loss = test_autoencoder_dataloader(device, model, dataloader_train, 
                                                                     shapedata, worst_path=worst_mesh_path, 
                                                                     worst_face_num=args['worst_face_num'],
                                                                     mm_constant = 100)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)   
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

loading checkpoint from file /home/jingwang/Data/data/FaceWarehouse/results/fully_connected_simple_autoencoder/latent_128/checkpoints/align_pose/checkpoint.pth.tar


100%|██████████| 290/290 [00:18<00:00, 15.86it/s]


('loss for ', 4407, tensor(0.7333, device='cuda:0'))
('loss for ', 7990, tensor(0.7386, device='cuda:0'))
('loss for ', 4652, tensor(0.7499, device='cuda:0'))
('loss for ', 4229, tensor(0.7571, device='cuda:0'))
('loss for ', 9272, tensor(0.8053, device='cuda:0'))
('loss for ', 5009, tensor(0.7569, device='cuda:0'))
('loss for ', 5751, tensor(0.7895, device='cuda:0'))
('loss for ', 3968, tensor(0.7604, device='cuda:0'))
('loss for ', 8612, tensor(0.7764, device='cuda:0'))
('loss for ', 4689, tensor(0.8434, device='cuda:0'))
('loss for ', 2876, tensor(0.8575, device='cuda:0'))
('loss for ', 8032, tensor(0.7779, device='cuda:0'))
('loss for ', 2945, tensor(0.7627, device='cuda:0'))
('loss for ', 6959, tensor(0.8138, device='cuda:0'))
('loss for ', 1443, tensor(0.9869, device='cuda:0'))
('loss for ', 988, tensor(0.8157, device='cuda:0'))
('loss for ', 2040, tensor(0.8673, device='cuda:0'))
('loss for ', 2952, tensor(0.7869, device='cuda:0'))
('loss for ', 8520, tensor(0.8912, device='cuda

In [55]:
if args['mode'] == 'test': # test with val set
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    predictions, norm_l1_loss, l2_loss = test_autoencoder_dataloader(device, model, dataloader_val, 
                                                                     shapedata, worst_path=worst_mesh_path, 
                                                                     worst_face_num=args['worst_face_num'],
                                                                     mm_constant = 100)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)   
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

100%|██████████| 2/2 [00:00<00:00, 186.53it/s]

loading checkpoint from file /home/jingwang/Data/data/FaceWarehouse/results/fully_connected_pca_autoencoder/latent_16/checkpoints/neutral/checkpoint.pth.tar
loss for  7 tensor(1.1442, device='cuda:0')
loss for  5 tensor(1.4197, device='cuda:0')
loss for  0 tensor(1.4422, device='cuda:0')
loss for  2 tensor(1.5218, device='cuda:0')
loss for  3 tensor(1.6134, device='cuda:0')
loss for  2 tensor(1.7671, device='cuda:0')
loss for  6 tensor(1.6526, device='cuda:0')
loss for  1 tensor(1.6962, device='cuda:0')
loss for  3 tensor(1.6305, device='cuda:0')
loss for  4 tensor(1.6477, device='cuda:0')
autoencoder: normalized loss 0.24331627786159515
autoencoder: euclidean distance in mm= 1.5535428524017334


In [10]:
# from optimize_to_get_result import optimize_to_get_result

import torch
import torch.optim as optim
import torch.nn as nn


def optimize_to_get_result(model, loss_fun, z_dim, device, targets, n_iter=1, output_loss=True):
    model.eval()
    
#     return torch.empty(0), model(targets)
    
    model.only_encode(True)
    z = model(targets)
    model.only_encode(False)
    
    model.only_decode(True)
    z_param = nn.Parameter(z)
    optimizer = optim.LBFGS(params=[z_param])
    for it in range(n_iter):
        def closure():
            optimizer.zero_grad()
            outputs = model(z_param)
            outputs = outputs.reshape(targets.shape) # decode, don't know shape
            loss = loss_fun(outputs, targets)
            loss.backward()
            if output_loss:
                print('loss ', loss.item())
            return loss
        optimizer.step(closure)
    outputs = model(z_param)
    outputs = outputs.reshape(targets.shape)
    
    model.only_decode(False)
    return z_param.data, outputs

def work_optimize():

    test_dataset = dataset_val
    # test_ids = [402,560, 50,  537, 390,
    #             364, 169, 616, 341, 590,
    #             499, 389,125, 370, 614,
    #             600, 194, 8, 653, 559]
#     test_ids = [602,194,560,390,402,537,625,8, 628,559,626,370,125,389,364,169,499,341,50]
    test_ids = list(range(10))
    inputs = torch.cat([test_dataset[idx]['points'].to(device).unsqueeze(0) for idx in test_ids])
    inputs = inputs.to(device)

    shapedata_mean = torch.Tensor(shapedata.mean).to(device)
    shapedata_std = torch.Tensor(shapedata.std).to(device)

    z, outputs = optimize_to_get_result(model, loss_fn, args['nz'], device,
                          inputs)
    from pprint import pprint
    pprint(z)

    outputs = outputs[:,:-1]
    inputs = inputs[:,:-1]

    old_outputs = outputs
    old_inputs = inputs

    outputs = outputs * shapedata_std + shapedata_mean
    inputs = inputs * shapedata_std + shapedata_mean

    per_l2_loss = torch.sqrt(torch.sum((outputs - inputs)**2, dim=2))

    for i in range(len(test_ids)):
        print(test_ids[i], torch.mean(per_l2_loss[i]))

    shapedata.save_meshes(os.path.join(worst_mesh_path, 'opt_input'),old_inputs.detach().cpu().numpy(),test_ids)
    shapedata.save_meshes(os.path.join(worst_mesh_path, 'opt_output'),old_outputs.detach().cpu().numpy(),test_ids)

work_optimize()

torch.Size([10, 11511, 3])
loss  0.24329513311386108
loss  0.24329493939876556
loss  0.24329471588134766
loss  0.24329450726509094
loss  0.24329429864883423
loss  0.2432941049337387
loss  0.2432938665151596
loss  0.24329368770122528
loss  0.24329344928264618
loss  0.24329324066638947
loss  0.24329303205013275
loss  0.24329283833503723
loss  0.24329259991645813
loss  0.2432924211025238
loss  0.2432921975851059
loss  0.243291974067688
loss  0.24329176545143127
loss  0.24329157173633575
loss  0.24329133331775665
loss  0.24329115450382233
tensor([[-2.1633e+01,  2.0553e+01,  4.9337e+01,  1.6851e+00,  2.3712e+01,
         -3.7888e+01, -1.4641e+01, -3.7942e+01,  3.1905e+01, -2.4031e+01,
          1.8875e+01, -3.5214e+01, -2.7131e+01,  3.2612e+01,  7.3559e+00,
         -9.1403e-01],
        [ 5.0887e+00, -4.2406e+00, -1.8859e+01,  4.0508e+01,  3.1154e+01,
         -6.6240e+01, -2.3830e+01, -6.9777e+01,  4.2486e+01,  1.3714e+00,
          3.7937e+01,  2.1957e+00,  4.2534e+01,  3.6115e+00, -3.74